# <center> <span style="color:Blue">Programacion 2</span> </center>
<center> <span style="color:Gray">  Challenge 2: Analizing Comments on Glassdoor </span>  </center>
<center> <span style="color:Gray"> Ilse Arredondo Reyes. No. Alumno 323019078</span>  </center>

### Project Stages  

**Stage 1: Web Scrapping**  

Will create a pipeline to extract Information of website Glassdoor
○ Web Scraping
This endpoint will receive as input Web scrapping:
- Open web page
- Read the content
- Extract content
- Save all in a data frame

----

**Stage 2: Model Creation**

🔹 **Text Preprocessing** 

This endpoint will receive as input Web scrapping result in a data frame
- Create English-Spanish data frame
- Analyze Separately Spanish and English data frame (Corpus)
- Data cleaning
- Stop words.
- Lemmatization
- N-grmas Distributions

🔹 **Classification proposed**

This endpoint will receive as input parameters of text preprocessing:

- Construction model
- End Date (date time)
- Taring model
- Calculation of grammatical probabilities

🔹 **Extraction of main features**
- Classification
- Sentimental Analysis (pysentimiento vs vader)

---

**Stage 3: Create a pipeline to MLOps**

🔹 **The MLOps part will be done with mlflow performing the following tasks**

- **Log metrics**
- **Model signatures**
- **Save the plot and log it as an artifact**
- **Tracking url (localhost)**
- **Run MLOps**



In [1]:
# Import data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.sklearn

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_glassdoor(url):
    # Setup Selenium options (headless)
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)
    
    try:
        # Open the Glassdoor page
        driver.get(url)
        time.sleep(5)  # wait for page to load completely
        
        # Read page content
        html = driver.page_source
        
        # Parse with BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')
        
        # Example: Extract job titles (customize selectors based on Glassdoor structure)
        job_titles = [tag.get_text(strip=True) for tag in soup.select('a.jobLink span')]
        companies = [tag.get_text(strip=True) for tag in soup.select('div.jobHeader span')]
        locations = [tag.get_text(strip=True) for tag in soup.select('span.subtle.loc')]
        
        # Combine data into a DataFrame
        data = pd.DataFrame({
            'Job Title': job_titles,
            'Company': companies,
            'Location': locations
        })
        
        return data
    
    finally:
        driver.quit()

# Example usage
url = 'https://www.glassdoor.com/Job/software-engineer-jobs-SRCH_KO0,17.htm'
df = scrape_glassdoor(url)
print(df.head())


Empty DataFrame
Columns: [Job Title, Company, Location]
Index: []


In [ ]:
!pip install spacy

In [11]:
!python -m spacy download en_core_web_sm
!python -m spacy download es_core_news_sm

c:\Users\ilse-\anaconda3\envs\PROGRAMACION2\python.exe: No module named spacy
c:\Users\ilse-\anaconda3\envs\PROGRAMACION2\python.exe: No module named spacy


In [12]:
import pandas as pd
import re
from langdetect import detect
import spacy
from nltk.corpus import stopwords
from nltk.util import ngrams
from collections import Counter

# Load language models
nlp_en = spacy.load("en_core_web_sm")
nlp_es = spacy.load("es_core_news_sm")

# Ensure NLTK stopwords are downloaded
import nltk
nltk.download("stopwords")

stopwords_en = set(stopwords.words('english'))
stopwords_es = set(stopwords.words('spanish'))

# Language detection
def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

# Clean text
def clean_text(text):
    text = re.sub(r'\W+', ' ', text)
    text = text.lower().strip()
    return text

# Lemmatization and stopword removal
def preprocess_text(text, lang):
    if lang == "en":
        doc = nlp_en(text)
        tokens = [token.lemma_ for token in doc if token.lemma_ not in stopwords_en and not token.is_punct]
    elif lang == "es":
        doc = nlp_es(text)
        tokens = [token.lemma_ for token in doc if token.lemma_ not in stopwords_es and not token.is_punct]
    else:
        return []
    return tokens

# Extract n-grams
def extract_ngrams(tokens, n=2):
    return list(ngrams(tokens, n))

# Apply pipeline
def preprocess_dataframe(df, text_column='Job Title'):
    df['Clean Text'] = df[text_column].fillna('').apply(clean_text)
    df['Lang'] = df['Clean Text'].apply(detect_language)
    df['Tokens'] = df.apply(lambda row: preprocess_text(row['Clean Text'], row['Lang']), axis=1)
    df['Bigrams'] = df['Tokens'].apply(lambda x: extract_ngrams(x, 2))
    return df


ModuleNotFoundError: No module named 'spacy'